In [13]:
import pandas as pd
import numpy as np
import pickle
import statistics
import os
from scipy.optimize import curve_fit
from tqdm import tqdm
import frgtools.misc as frgm


import matplotlib as mpl
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
from matplotlib import style
mpl.rcParams.update(mpl.rcParamsDefault)

# Define and Plot Si Responsivity

In [14]:
from scipy import interpolate
resp = pd.read_csv('/Volumes/GoogleDrive/Shared drives/FenningLab2/StaffFolders/Connor Dolan/PLQY/Si Responsivity.csv')
f = interpolate.interp1d(resp['Wavelength'],resp['Responsivity'])
xnew = range(320,1100)
ynew = f(xnew)

In [15]:
# plt.plot(xnew,ynew)

# #plt.title('Si Detector Responsivity')
# #plt.ylabel('Responsivity')
# #plt.xlabel('Wavelength')
# plt.show()

# Load Data, Convert to Values of Interest

In [16]:
rootdir = os.path.join('/Volumes/GoogleDrive/My Drive/Characterization_RAWDATA/PLQY/20210819_T24') #replace this with the filepath to root directory for the experiment
bg = os.path.join(rootdir, 'bg')
raw = os.path.join(rootdir, 'raw')
extra = os.path.join(rootdir, 'extra')
tseries = os.path.join(rootdir, 'tseries')

In [17]:
fids = []
for f in frgm.listdir(rootdir, display = False):
    if 'control' in f and '_rgb' not in f:
        continue
    fids.append(f)
fids

['/Volumes/GoogleDrive/My Drive/Characterization_RAWDATA/PLQY/20210819_T24/a_mptio2_plqy',
 '/Volumes/GoogleDrive/My Drive/Characterization_RAWDATA/PLQY/20210819_T24/d_glass_plqy',
 '/Volumes/GoogleDrive/My Drive/Characterization_RAWDATA/PLQY/20210819_T24/b_glass_plqy',
 '/Volumes/GoogleDrive/My Drive/Characterization_RAWDATA/PLQY/20210819_T24/d_ctio2_plqy',
 '/Volumes/GoogleDrive/My Drive/Characterization_RAWDATA/PLQY/20210819_T24/a_glass_plqy',
 '/Volumes/GoogleDrive/My Drive/Characterization_RAWDATA/PLQY/20210819_T24/c_glass_plqy',
 '/Volumes/GoogleDrive/My Drive/Characterization_RAWDATA/PLQY/20210819_T24/a_ctio2_plqy',
 '/Volumes/GoogleDrive/My Drive/Characterization_RAWDATA/PLQY/20210819_T24/b_ctio2_plqy',
 '/Volumes/GoogleDrive/My Drive/Characterization_RAWDATA/PLQY/20210819_T24/c_ctio2_plqy']

In [21]:
for n in range(len(fids)):
    data = pickle.load(open(fids[n],'rb'))
 
    results = {
            n*6+0: {'name':fids[n].split('/')[::-1][0].split('_plqy')[0],'optics':'0b_0lp', 'mean':data[0]['IntSphere']['Mean']/data[0]['Reference']['Mean']},
            n*6+1: {'name':fids[n].split('/')[::-1][0].split('_plqy')[0],'optics':'1b_0lp', 'mean':data[1]['IntSphere']['Mean']/data[1]['Reference']['Mean']},
            n*6+2: {'name':fids[n].split('/')[::-1][0].split('_plqy')[0],'optics':'1b_1lp', 'mean':data[2]['IntSphere']['Mean']/data[2]['Reference']['Mean']},
            n*6+3: {'name':fids[n].split('/')[::-1][0].split('_plqy')[0],'optics':'0b_1lp', 'mean':data[3]['IntSphere']['Mean']/data[3]['Reference']['Mean']},
            n*6+4: {'name':fids[n].split('/')[::-1][0].split('_plqy')[0],'optics':'ns_1lp', 'mean':data[4]['IntSphere']['Mean']/data[4]['Reference']['Mean']},
            n*6+5: {'name':fids[n].split('/')[::-1][0].split('_plqy')[0],'optics':'ns_0lp',  'mean':data[5]['IntSphere']['Mean']/data[5]['Reference']['Mean']},
        }
    if n ==0:
        df = pd.DataFrame(results).T
        
    if n>0:
        df_temp = pd.DataFrame(results).T
        df=df.append(df_temp)

In [22]:
df_calc = pd.DataFrame(columns=['PLQY'])

# data_names = 

for n in range(len(df['name'].unique())):
    df_calc=df_calc.append(pd.Series(df['name'].unique()[n]), ignore_index=True)
df_calc = df_calc.rename(columns={'PLQY':'PLQY', 0:'name'})


df_calc = df_calc[['name', 'PLQY']]


#     = df['name'].unique()[n]

In [23]:
# for n in range(df_calc.shape[0]):
laser_wavelength = 532
emission_wavelength = 677
laser_responsivity = ynew[laser_wavelength-320]
emission_responsivity = ynew[emission_wavelength-320]

for q in range(len(df['name'].unique())):
#         df.loc[df['name']==df['name'].unique()[q]]

    outBeam_outLP = float(df.loc[df['name']==df['name'].unique()[q]].loc[df.loc[df['name']==df['name'].unique()[q]]['optics']=='0b_0lp']['mean'])
    inBeam_outLP = float(df.loc[df['name']==df['name'].unique()[q]].loc[df.loc[df['name']==df['name'].unique()[q]]['optics']=='1b_0lp']['mean'])
    inBeam_inLP = float(df.loc[df['name']==df['name'].unique()[q]].loc[df.loc[df['name']==df['name'].unique()[q]]['optics']=='1b_1lp']['mean'])
    outBeam_inLP = float(df.loc[df['name']==df['name'].unique()[q]].loc[df.loc[df['name']==df['name'].unique()[q]]['optics']=='0b_1lp']['mean'])
    ns_inLP = float(df.loc[df['name']==df['name'].unique()[q]].loc[df.loc[df['name']==df['name'].unique()[q]]['optics']=='ns_1lp']['mean'])
    ns_outLP = float(df.loc[df['name']==df['name'].unique()[q]].loc[df.loc[df['name']==df['name'].unique()[q]]['optics']=='ns_0lp']['mean'])

    E_in = inBeam_inLP
    E_out = outBeam_inLP
    X_out = outBeam_outLP - outBeam_inLP
    X_in = inBeam_outLP - inBeam_inLP
    X_empty = ns_outLP - ns_inLP

    E_in = E_in / emission_responsivity
    E_in = E_in / emission_wavelength
    E_out = E_out / emission_responsivity
    E_out = E_out / emission_wavelength
    X_in = X_in / laser_responsivity
    X_in = X_in / laser_wavelength
    X_out = X_out / laser_responsivity
    X_out = X_out / laser_wavelength
    X_empty = X_empty / laser_responsivity
    X_empty = X_empty / laser_wavelength
    a = (X_out-X_in)/X_out
    plqy = (E_in-(1-a)*E_out)/(X_empty*a)

    df_calc['PLQY'][q]= plqy

In [24]:
df_calc = df_calc.sort_values(by=['PLQY'],  ascending=False)

df_calc = df_calc.reset_index()
df_calc = df_calc[['name', 'PLQY']]
df_calc

,name,PLQY
0,d_ctio2,0.00734386
1,c_ctio2,0.00700002
2,a_ctio2,0.00695932
3,b_ctio2,0.00689955
4,a_mptio2,0.00687352
5,b_glass,0.00662795
6,d_glass,0.00658978
7,a_glass,0.00626588
8,c_glass,0.00626551


In [25]:
df_calc['Chlorine_Loading'] = ''
df_calc['Substrate'] = ''



name_key = {
    'd': '0% Cl',
    'c': '1% Cl',
    'b': '5% Cl',
    'a': '10% Cl',
    'ctio2': 'C-TiO2',
    'mptio2': 'MP-TiO2',
    'glass': 'Glass'
}

def getname(n):
    letter = n.split('_')[0]
    return name_key[letter]

def getname1(n):
    letter = n.split('_')[1]
    return name_key[letter]

df_calc['Chlorine_Loading'] = df_calc['name'].apply(getname)
df_calc['Substrate'] = df_calc['name'].apply(getname1)
# df = df.sort_values(by=['label'])
# df = df.reset_index(drop=True)

In [26]:
df_calc = df_calc[['Chlorine_Loading', 'Substrate', 'PLQY']]

In [31]:
df_calc = df_calc.rename(columns={'Chlorine_Loading':'Chlorine_Loading', 'Substrate':'Substrate', 'PLQY': 'PLQY 1=100%'})

In [32]:
df_calc

,Chlorine_Loading,Substrate,PLQY 1=100%
0,0% Cl,C-TiO2,0.00734386
1,1% Cl,C-TiO2,0.00700002
2,10% Cl,C-TiO2,0.00695932
3,5% Cl,C-TiO2,0.00689955
4,10% Cl,MP-TiO2,0.00687352
5,5% Cl,Glass,0.00662795
6,0% Cl,Glass,0.00658978
7,10% Cl,Glass,0.00626588
8,1% Cl,Glass,0.00626551
